<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare</a></span><ul class="toc-item"><li><span><a href="#Import-Library" data-toc-modified-id="Import-Library-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import Library</a></span></li><li><span><a href="#Fixed-Point" data-toc-modified-id="Fixed-Point-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Fixed Point</a></span></li><li><span><a href="#Newton's-Method" data-toc-modified-id="Newton's-Method-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Newton's Method</a></span></li><li><span><a href="#Steepest-Descent" data-toc-modified-id="Steepest-Descent-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Steepest Descent</a></span></li></ul></li><li><span><a href="#-Run-" data-toc-modified-id="-Run--2"><span class="toc-item-num">2&nbsp;&nbsp;</span><font color="orange"> Run <font></font></font></a></span><ul class="toc-item"><li><span><a href="#Fixed-Point" data-toc-modified-id="Fixed-Point-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Fixed Point</a></span></li><li><span><a href="#Newton's-method" data-toc-modified-id="Newton's-method-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Newton's method</a></span></li><li><span><a href="#Steepest-Descent" data-toc-modified-id="Steepest-Descent-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Steepest Descent</a></span></li></ul></li></ul></div>

You can directly go to **Chapter 2 (Run)** to look experiment results.

# Prepare

## Import Library

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -5)

## Fixed Point

In [2]:
class FixedPointMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [3]:
class FixedPoint(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.cos(x[1] * x[2]) / 3.0 + 1.0 / 6.0
        sol[1] = math.sqrt(x[0] * x[0] + math.sin(x[2]) + 1.06) / 9.0 - 0.1
        sol[2] = -math.exp(-x[0] * x[1]) / 20.0 - (10.0 * math.pi - 3.0) / 60.0
        return sol

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [4]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.cos(x[1] * x[2]) / 3.0 + 1.0 / 6.0
        sol[1] = math.sqrt(sol[0] * sol[0] + math.sin(x[2]) + 1.06) / 9.0 - 0.1
        sol[2] = -math.exp(-sol[0] * sol[1]) / 20.0 - (10.0 * math.pi - 3.0) / 60.0
        return sol

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

## Newton's Method

In [5]:
class NewtonMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [6]:
class Newton(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1.0 / 2.0
        sol[1] = pow(x[0], 2) - 81 * pow(x[1] + 0.1, 2) + math.sin(x[2]) + 1.06
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3.0) / 3.0
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3.0
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -162 * (x[1] + 0.1)
        jac[1][2] = math.cos(x[2])
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            jac = self.jacobian(x)
            f = -self.f(x)
            y = linalg.solve(jac, f)
            nx = x + y
            residual = linalg.norm(x - nx, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

## Steepest Descent

In [7]:
class SteepestDescentMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def g(self, x):
        return NotImplementedError('Implement g()!')

    @abstractmethod
    def grad_g(self, x):
        return NotImplementedError('Implement grad_g()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [8]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1.0 / 2.0
        sol[1] = pow(x[0], 2) - 81 * pow(x[1] + 0.1, 2) + math.sin(x[2]) + 1.06
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3.0) / 3.0
        return sol

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3.0
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -162 * (x[1] + 0.1)
        jac[1][2] = math.cos(x[2])
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][5] = linalg.norm(xk - x, np.inf)

        return df

# <font color='orange'> Run <font>

## Fixed Point

In [9]:
pd.options.display.float_format = '{:,.8f}'.format

In [10]:
x0 = np.array([0.1, 0.1, -0.1])
FixedPoint().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.10000000,0.10000000,-0.10000000,nan,0.42359877
1,0.49998333,0.00944115,-0.52310127,0.42310127,0.00944113
2,0.49999593,0.00002557,-0.52336331,0.00941558,0.00023546
3,0.50000000,0.00001234,-0.52359814,0.00023483,0.00001232
4,0.50000000,0.00000003,-0.52359847,0.00001230,0.00000031
5,0.50000000,0.00000002,-0.52359877,0.00000031,0.00000000


In [11]:
x0 = np.array([0.1, 0.1, -0.1])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.10000000,0.10000000,-0.10000000,nan,0.42359878
1,0.49998333,0.02222979,-0.52304613,0.42304613,0.02222979
2,0.49997747,0.00002815,-0.52359807,0.02220164,0.00002815
3,0.50000000,0.00000004,-0.52359877,0.00002812,0.00000004
4,0.50000000,0.00000000,-0.52359878,0.00000004,0.00000000


## Newton's method

In [12]:
pd.options.display.float_format = '{:,.10f}'.format

In [13]:
x0 = np.array([0.1, 0.1, -0.1])
Newton().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.1000000000,0.1000000000,-0.1000000000,nan,0.4235987756
1,0.4998696729,0.0194668485,-0.5215204719,0.4215204719,0.0194668485
2,0.5000142402,0.0015885914,-0.5235569643,0.0178782572,0.0015885914
3,0.5000001135,0.0000124448,-0.5235984501,0.0015761466,0.0000124448
4,0.5000000000,0.0000000008,-0.5235987756,0.0000124440,0.0000000008
5,0.5000000000,0.0000000000,-0.5235987756,0.0000000008,0.0000000000


## Steepest Descent

In [14]:
pd.options.display.float_format = '{:,.6f}'.format
x0 = np.array([0, 0, 0])
SteepestDescent().run(x0)

,x1,x2,x3,g,residual,actual-residual
0,0.000000,0.000000,0.000000,111.974771,nan,0.523546
1,0.011218,0.010096,-0.522741,2.327617,0.522741,0.485544
2,0.137860,-0.205453,-0.522059,1.274058,0.215549,0.358902
3,0.266959,0.005511,-0.558494,1.068131,0.210964,0.229803
4,0.272734,-0.008118,-0.522006,0.468309,0.036488,0.224028
5,0.308689,-0.020403,-0.533112,0.381087,0.035956,0.188073
6,0.314308,-0.014705,-0.520923,0.318837,0.012188,0.182454
7,0.324267,-0.008525,-0.528431,0.287024,0.009958,0.172495
8,0.330809,-0.009678,-0.520662,0.261579,0.007768,0.165953
9,0.339809,-0.008592,-0.528080,0.238486,0.009000,0.156953
